3. Prepare the whole pipeline
    1. Data augmentation
        - First take the unaugmented original dataset and proceed
        -  Augment the data with a predefined seed for each of the following techniques: rotation, flipping, contrast, brightness change, random erasing
    2. Choose hyperparameters
        - For the hyper-parameters related to the training process we chose batch size, learning rate, and number of epochs
        - For the hyper-parameters related to Regularization we decided to use L2 Regularization (Weight Decay) and Dropout Rate
    3. Train each of the prepared models on the augmented dataset for a chosen augmentation technique
    4. Test and collect data regarding models’ performance on the augmented dataset
    5. Repeat several times (>=3) from 3.
    6. Choose different values for hyperparameters and start from 3.
    7. Choose the next augmentation technique and start from 2.
    8. Repeat the process starting from 1. several times (>=3) with a different seed each time

In [2]:
import os
from enum import Enum
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
import cv2
import random


import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD, Lion
from keras.regularizers import L2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from efficientnet.tfkeras import EfficientNetB0
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Move 5400 images from each class from valid to train
 - There is a safety check, if it has been already done it won't do it again :)

In [3]:
rootdir = './Dataset/valid'

for subdir, dirs, files in os.walk(rootdir):
    if subdir != rootdir:
        for subsubdir, subdirs, files in os.walk(subdir):
            if len(files) < 5400:
                break;
            for i in range(5400):
                os.rename(os.path.join(os.path.join("./Dataset/valid",os.path.basename(subsubdir)),files[i]), os.path.join(os.path.join("./Dataset/train",os.path.basename(subsubdir)),files[i]))

## Pipeline

Classes

In [4]:
class AugmentationTechnique(Enum):
    NoAugmentation = 0
    Rotation = 1
    Flipping = 2
    Contrast = 3
    Brightness = 4
    RandomErasing = 5

class ModelType(Enum):
    MobileNet =1
    EfficientNet = 2

class OptimizerType(Enum):
    Adam = 1
    Sgd=2
    Lion=3
class RegularizationType(Enum):
    NoRegularization = 1
    L2 =2
    WeightDecay=3
def getRegularizer(regularizerType,value):
    match regularizerType:
        case RegularizationType.L2:
            return L2(value)
def getDenseLayer(model,regularization, activation,nodes):
        match regularization['type']:
            case RegularizationType.WeightDecay:
                model.add(Dropout(rate=regularization['value']))
                model.add(Dense(nodes, activation=activation))
            case RegularizationType.NoRegularization:
                model.add(Dense(nodes, activation=activation))
            case RegularizationType.L2:
                model.add(Dense(nodes, activation=activation, kernel_regularizer=getRegularizer(type=regularization['type'],value=regularization['value'])))

class Model:
    def fit(self,batch_size,epochs,train_data_generator,valid_data_generator):
        pass
    def predict(self,test_data_generator):
        pass
    def __init__(self,optimizer,loss,metrics):
        pass

# TODO: implement 3rd model
    # TODO: How should we handle Droput and L2, should it be applied to each Dense layer?
class CustomMobileNetModel(Model):
    def __init__(self,optimizer,loss,metrics,regularizer):
        # Load MobileNetV3Large without the top classification layer
        base_model = MobileNetV3Large(include_top=False, weights='imagenet', input_shape=(32, 32, 3))

        # Freeze the base model layers
        base_model.trainable = False

        # Add additional layers on top of MobileNetV3Large
        model = tf.keras.Sequential()
        model.add(base_model)
        model.add(GlobalAveragePooling2D())

        getDenseLayer(model=model,regularization=regularizer,activation='relu',nodes=1024)
        model.add(BatchNormalization())
        getDenseLayer(model=model,regularization=regularizer,activation='relu',nodes=512)
        model.add(BatchNormalization())
        getDenseLayer(model=model,regularization=regularizer,activation='softmax',nodes=10)
            # Output layer with size 10 for classification
        # Compile the model
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        self.model=model

    def fit(self,batch_size,epochs,train_data_generator,valid_data_generator):
        self.model.fit(
        train_data_generator,
        steps_per_epoch=batch_size,
        epochs=epochs,
        validation_data=valid_data_generator,
        validation_steps=batch_size
        )
    def predict(self,test_data_generator):
        return self.model.predict(test_data_generator ,steps=len(test_data_generator))
    
class CustomEfficientNetModel(Model):
    def __init__(self,optimizer,loss,metrics, regularizer):
        # Load MobileNetV3Large without the top classification layer
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
        
        # Freeze the base model layers
        base_model.trainable = False
        # Add additional layers on top of EfficientNet
        model = tf.keras.Sequential()
        model.add(base_model)
        model.add(GlobalAveragePooling2D())
        getDenseLayer(model=model,regularization=regularizer,activation='relu',nodes=1024)
        model.add(BatchNormalization())
        getDenseLayer(model=model,regularization=regularizer,activation='relu',nodes=512)
        model.add(BatchNormalization())
        getDenseLayer(model=model,regularization=regularizer,activation='softmax',nodes=10)
            # Output layer with size 10 for classification
        # Compile the model
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        self.model=model
    def fit(self,batch_size,epochs,train_data_generator,valid_data_generator):
        self.model.fit(
        train_data_generator,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=valid_data_generator,
        )
    def predict(self,test_data_generator):
        return self.model.predict(test_data_generator)


Helper functions

In [5]:
# https://github.com/yu4u/cutout-random-erasing
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        if input_img.ndim == 3:
            img_h, img_w, img_c = input_img.shape
        elif input_img.ndim == 2:
            img_h, img_w = input_img.shape

        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            if input_img.ndim == 3:
                c = np.random.uniform(v_l, v_h, (h, w, img_c))
            if input_img.ndim == 2:
                c = np.random.uniform(v_l, v_h, (h, w))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w] = c

        return input_img

    return eraser


In [6]:

def createModel(modelType, optimizer, loss, metrics, regularizer)->Model:
    match modelType:
        case ModelType.MobileNet:
            return CustomMobileNetModel(optimizer=optimizer,loss=loss,metrics=metrics,regularizer=regularizer)
        case ModelType.EfficientNet:
            return NotImplementedError

def set_seed(seed=0):
    np.random.seed(seed) 
    tf.random.set_seed(seed) 
    random.seed(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = "1"
    os.environ['TF_CUDNN_DETERMINISM'] = "1"
    os.environ['PYTHONHASHSEED'] = str(seed)
def getContrastChange():
    def contrastChange(input_img):
        contrast = np.randint(0,100)
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)

        input_img = cv2.addWeighted(input_img, alpha_c, input_img, 0, gamma_c)
        return input_img
    return contrastChange


def augumentData(technique, seed):
    match technique:
        case AugmentationTechnique.NoAugmentation:
            return ImageDataGenerator()
        case AugmentationTechnique.Rotation:
            return ImageDataGenerator(rotation_range=20)
        case AugmentationTechnique.Flipping:
            return ImageDataGenerator(horizontal_flip=True,vertical_flip=True)
        case AugmentationTechnique.Brightness:
            return ImageDataGenerator(brightness_range=[0.2,1.8])
        case AugmentationTechnique.Contrast:
            return ImageDataGenerator(preprocessing_function=getContrastChange())
        case AugmentationTechnique.RandomErasing:
            return ImageDataGenerator(preprocessing_function=get_random_eraser(v_l=0, v_h=0.5))

def getAccuracy(y_result, y_test):
    correct_amount =0 
    for i, result in enumerate(y_result):
        if result == y_test[i]:
            correct_amount+=1
    return correct_amount/len(y_test)

def getOptimizer(type, learningRate):
    match type:
        case OptimizerType.Adam:
            return Adam(learning_rate=learningRate)
        case OptimizerType.Sgd:
            return SGD(learning_rate=learningRate)
        case OptimizerType.Lion:
            return Lion(learning_rate=learningRate)
            
        

# Arrays containing different hyper-parameter values

In [10]:
train_dir = 'Dataset/train'
valid_dir = 'Dataset/valid'
test_dir = 'Dataset/test'
image_size = (32, 32)
# training process
# batchSizes =[64,128,256]
batchSizes = [64]
#learningRates = [0.001, 0.01,0.1]
learningRates=[0.1]
#numberOfEpochs =[5,10,15]
numberOfEpochs=[5]

# regularization
# TODO: I guess we should add more variants of these
#regularizers = [{"type":RegularizationType.NoRegularization,"value":0},{"type":RegularizationType.WeightDecay,"value":0.5},{"type":RegularizationType.L2,"value":0.01}]
regularizers=[{"type":RegularizationType.WeightDecay,"value":0.5}]

# augmentation
# augmentationTechniques =[AugmentationTechnique.NoAugmentation,AugmentationTechnique.Rotation,AugmentationTechnique.Flipping,AugmentationTechnique.Contrast,AugmentationTechnique.Brightness,AugmentationTechnique.RandomErasing]
augmentationTechniques=[AugmentationTechnique.RandomErasing]
# seeds = [123,42,9]
seeds = [123]

Main pipeline loop

In [ ]:
def performExperiment(modelType):
    # TODO: Refactor to waterfall like approach
    results = []
    valid_datagen = ImageDataGenerator(rescale=1./255)
    for batchSize in batchSizes:
        for learningRate in learningRates:
            for epochNumber in numberOfEpochs:
                for augmentation in augmentationTechniques:
                    for regularizer in regularizers:
                        for seed in seeds:
                            set_seed(seed)

                            train_augmented_data_generator = augumentData(technique=augmentation,seed=seeds[0])

                            train_generator = train_augmented_data_generator.flow_from_directory(
                                train_dir,
                                target_size=image_size,
                                batch_size=batchSize,
                                class_mode='categorical'
                            )

                            valid_generator = valid_datagen.flow_from_directory(
                                valid_dir,
                                target_size=image_size,
                                batch_size=batchSize,
                                class_mode='categorical'
                            )

                            test_generator = valid_datagen.flow_from_directory(
                                test_dir,
                                target_size=image_size,
                                batch_size=batchSize,
                                class_mode='categorical'
                            )
                            # create the model

                            model = createModel(modelType=modelType,regularizer=regularizer,optimizer=getOptimizer(type=OptimizerType.Sgd,learningRate=learningRate), loss='categorical_crossentropy', metrics=['accuracy'])
                            # train the model 
                            model.fit(train_data_generator=train_generator,batch_size=batchSize,epochs=epochNumber,valid_data_generator=valid_generator)

                            # get accuracy
                            y_pred = model.predict(test_generator)
                            # Convert probabilities to class labels
                            predicted_classes = np.argmax(y_pred, axis=1)

                            # Get true class labels
                            true_classes = test_generator.classes
                            accuracy = getAccuracy(predicted_classes,true_classes)

                            # append to results
                            # It's probably easier to create a simple 2d array and then transform it to a dataframe
                            results.append([accuracy, augmentation,batchSize,learningRate,epochNumber])
                            # results.append({'accuracy':accuracy,'augmentation': augmentation,'batchSize':batchSize,'learningRate':learningRate,'numberOfEpochs':epochNumber})
    return results

In [11]:
performExperiment(modelType=ModelType.MobileNet)

Found 144000 images belonging to 10 classes.
Found 36000 images belonging to 10 classes.
Found 90000 images belonging to 10 classes.


c:\Users\prusak.patryk\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\applications\mobilenet_v3.py:512: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


Epoch 1/5


c:\Users\prusak.patryk\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


64/64 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.1312 - loss: 3.3051 - val_accuracy: 0.1006 - val_loss: 2.9658
Epoch 2/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 107s 2s/step - accuracy: 0.1674 - loss: 2.7242 - val_accuracy: 0.0950 - val_loss: 2.7898
Epoch 3/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 106s 2s/step - accuracy: 0.1699 - loss: 2.6327 - val_accuracy: 0.0989 - val_loss: 2.4239
Epoch 4/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - accuracy: 0.1672 - loss: 2.6094 - val_accuracy: 0.0989 - val_loss: 2.5790
Epoch 5/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.1708 - loss: 2.5273 - val_accuracy: 0.1001 - val_loss: 2.5137
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 1204s 855ms/step


[[0.09956666666666666, <AugmentationTechnique.RandomErasing: 5>, 64, 0.1, 5]]

# UNUSED

# Pipeline checking each combination of parameters (with 3 values for each parameter it should take approx 48h on google colab)

In [8]:
def performExperiment(modelType):
    results = []
    valid_datagen = ImageDataGenerator(rescale=1./255)
    for batchSize in batchSizes:
        for learningRate in learningRates:
            for epochNumber in numberOfEpochs:
                for augmentation in augmentationTechniques:
                    for regularizer in regularizers:
                        for seed in seeds:
                            set_seed(seed)

                            train_augmented_data_generator = augumentData(technique=augmentation,seed=seeds[0])

                            train_generator = train_augmented_data_generator.flow_from_directory(
                                train_dir,
                                target_size=image_size,
                                batch_size=batchSize,
                                class_mode='categorical'
                            )

                            valid_generator = valid_datagen.flow_from_directory(
                                valid_dir,
                                target_size=image_size,
                                batch_size=batchSize,
                                class_mode='categorical'
                            )

                            test_generator = valid_datagen.flow_from_directory(
                                test_dir,
                                target_size=image_size,
                                batch_size=batchSize,
                                class_mode='categorical'
                            )
                            # create the model

                            model = createModel(modelType=modelType,regularizer=regularizer,optimizer=getOptimizer(type=OptimizerType.Sgd,learningRate=learningRate), loss='categorical_crossentropy', metrics=['accuracy'])
                            # train the model 
                            model.fit(train_data_generator=train_generator,batch_size=batchSize,epochs=epochNumber,valid_data_generator=valid_generator)

                            # get accuracy
                            y_pred = model.predict(test_generator)
                            # Convert probabilities to class labels
                            predicted_classes = np.argmax(y_pred, axis=1)

                            # Get true class labels
                            true_classes = test_generator.classes
                            accuracy = getAccuracy(predicted_classes,true_classes)

                            # append to results
                            # It's probably easier to create a simple 2d array and then transform it to a dataframe
                            results.append([accuracy, augmentation,batchSize,learningRate,epochNumber])
                            # results.append({'accuracy':accuracy,'augmentation': augmentation,'batchSize':batchSize,'learningRate':learningRate,'numberOfEpochs':epochNumber})
    return results

In [ ]:


class HyperParameter(Enum):
    BatchSize=1
    LearningRate =2
    NumberOfEpochs =3

class HyperParameters:
    def __init__(self, batchSizes, learningRates, numberOfEpochs):
        self.batchSizes=batchSizes
        self.learningRates=learningRates
        self.numberOfEpochs=numberOfEpochs
        self.currentIndex =0
    
    def getNextHyperParameter(self):
        if self.currentIndex<len(self.batchSizes):
            return self.batchSizes[self.currentIndex], HyperParameter.BatchSize
        elif self.currentIndex<(len(self.batchSizes+self.learningRates)):
            return self.learningRates[self.currentIndex-len(self.batchSizes)], HyperParameter.LearningRate
        elif self.currentIndex<(len(self.batchSizes)+len(self.learningRates)+len(self.numberOfEpochs)):
            return self.num